In [ ]:
import numpy as np
from scipy import stats
import pandas as pd
from pathlib import Path
import re

rate_re = re.compile(r"(\(([\d.]+)%\))")
n_match = 1000
alpha = 0.05
z = stats.norm.ppf(1 - alpha / 2)
data = []

for file in Path("results/search_c").glob("*.txt"):
    C = float(file.stem)
    with file.open() as f:
        lines = f.readlines()
    # get the non-lose rate
    p = float(rate_re.search(lines[-1]).group(2)) / 100
    # get the confidence interval
    ci = z * np.sqrt(p * (1 - p) / n_match)
    data.append((C, p, ci))

df = pd.DataFrame(data, columns=["C", "p", "ci"])
df = df.sort_values("C")
df

In [ ]:
import matplotlib.pyplot as plt

X = np.arange(len(df))
plt.errorbar(X, df["p"], yerr=df["ci"], fmt="o-", capsize=3, ecolor="gray", label="Non-lose rate of UCT")
plt.xticks(X, labels=df["C"])
plt.xlabel("$C$")
plt.ylabel("$p$")
plt.title("Alpha-beta vs UCT")
plt.legend()
plt.tight_layout()
plt.savefig("results/search_c.svg")
plt.show()